In [ ]:
import torch
print(torch.__version__)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

定义参数

In [ ]:
model_name="Helsinki-NLP/opus-mt-en-cs"
source_lang="en"
target_lang="cs"
dataset_name="wmt16"

下载读取WMT数据集

In [ ]:
from datasets import load_dataset

# Download the WMT dataset
dataset = load_dataset(dataset_name, f"cs-en")
translation=dataset["test"]["translation"]
source_texts = [t[source_lang] for t in translation]  # Adjust the size as needed
reference_texts = [t[target_lang] for t in translation]

测试模型

In [ ]:
from transformers import pipeline
translation_pipeline = pipeline(task="translation", model=model_name,device=device)
translation_pipeline('hello world')

生成翻译句子

In [ ]:
import pandas as pd
translations = translation_pipeline(source_texts, src_lang=source_lang,tgt_lang=target_lang)
translations = [translation['translation_text'] for translation in translations]
# 输出翻译结果
df = pd.DataFrame({
    'Source': source_texts,
    'Translation': translations
})
df

使用sacrebleu评价模型

In [ ]:
import sacrebleu
bleu_score = sacrebleu.corpus_bleu(translations, [reference_texts]).score
f"BLEU Score: {bleu_score}"

写入文件

In [ ]:
with open("./BLEU_result.txt","a",encoding="utf-8") as f:
    f.write(f"\n{model_name} {source_lang} {target_lang} {bleu_score}")